## Project 1

In the STARR-seq analysis we performed Module 7, we ended by plotting boxplots for STARR-seq scores of each motif that was enriched in enhancers activated by ecdysone compared to constitutive enhancers. Write code to compare those scores against STARR-seq scores for the same motifs found in enhancers in the absence of ecdysone. This would be the output:
<img src='boxplot_compare.png'>

The suffix "_wo" means those boxplots correspond to enhancers with the corresponding motifs found in the STARR-seq dataset without ecdysone treatment.

In [ ]:
#PUT YOUR CODE FOR PROJECT 1 HERE

## Project 2

For the ecdysone responsive enhancers, we want to find out if adjacent genes change during ecdysone treatment. Gene expression changes upon addition of ecdysone is in the file "rpkm_ecdysone_to_ctl.txt", which has two columns separated by a tab. First column is Gene_name, second column named "Enrichment" is the log2 fold change of expression upon addition of ecdysone. The genes next to enhancers responsive to ecdysone are listed in "w_ecd_genes.list" and the control group is listed in "wo_ecd_genes.list". Make a boxplot of expression change comparing genes next to ecdysone enhancers and genes next to control enhancers. Your result should look like this:

<img src='gene_exp_boxplot.png'>

In [ ]:
#PUT YOUR CODE FOR PROJECT 2 HERE

## Project 3

RBFOX2 is a human splicing factor.  It regulates the inclusion of exons through binding to specific RNA sites near alternative exons.  Below is a "splicing map" from Jangi et al (2014) for RBFOX2.  It combines RBP/RNA interaction data (CLIP-seq) with data about which exons are more/less included upon RBFOX2 knockdown.

![RBFOXmap.png](RBFOXmap.png)

As you can see, exons whose inclusion is enhanced (green line) tend to have RBFOX2 bound in the intron immediately downstream of the exon.  This indicates that RBFOX2 likely promotes inclusion of this exon by binding in the downstream intron.  

We might wonder how RBFOX2 knows to bind these specific introns.  We know that RNA-binding proteins like RBFOX2 often recognize their RNA targets through interaction with short RNA sequences (kmers).  Perhaps, then, RBFOX2 is recognizing a specific kmer that is enriched in the introns downstream of the exons it enhances.  Can we figure out what this kmer (or kmers) might be?

One way to do this would be by comparing two groups of sequences.  One group could be sequences from introns downstream of exons whose inclusion is enhanced by RBFOX2.  Think of these as the "affected" exons.  The other group could be sequences from the same downstream intron, but only after exons whose inclusion is unaffected by RBFOX2. Think of these as the "control" exons.  

*As an aside, we know which exons are enhanced by RBFOX2 and which are not by analyzing splicing transcriptome-wide in cells that we treated with either RBFOX2 siRNA or control siRNA.*

We would expect that the kmer that RBFOX2 is recognizing would be enriched in introns downstream of "affected" exons relative to the introns downstream of "control" exons.  It just so happens that we have made fasta files containing intronic sequences downstream of both exon classes.  You can download them from the links below.

[Enhanced](https://github.com/srinivasramachandran/Anschutz_python/blob/master/DownstreamIntron.Enhanced.fasta)

[Control](https://github.com/srinivasramachandran/Anschutz_python/blob/master/DownstreamIntron.Control.fasta)

> For the splicing afficionados out there, these files are actually of the last 50 nt of each exon and the first 150 nt of the downstream intron.

What we would like to do is write code that compares the sequence content of the two files and identifies 5-mers that are enriched in the "Enhanced" file relative to the "control" file.  This can be done with the following approach:

1.  Go through each file and count the number of occurences of each 5-mer.  This might be best stored as a dictionary of the form {kmer : number of occurences}.
2.  Calculate a log2 enrichment for each for each kmer in the affected relative to control by dividing the frequency of the kmer in the affected sequences to its frequency in control sequences and taking the log base 2 of the ratio.  Remember that the frequency of a kmer is that number of occurences of that kmer divided by the number of occurences of all kmers.  Frequencies must be between 0 and 1.  Enrichments will be positive if the kmer is enriched in the enhanced sequences and negative if enriched in the control sequences.
3.  Calculate the statistical significance of that enrichment with a Fisher's exact test.  This can be done by importing the fisher_exact function from the scipy.stats module.  See the [documentation](http://lagrange.univ-lyon1.fr/docs/scipy/0.17.1/generated/scipy.stats.fisher_exact.html) for fisher_exact for help.
4.  Correct your Fisher's exact p-values for multiple hypothesis testing with a Bonferroni correction.  Very simply, multiply your original pvalue from fisher_exact by the number of kmers you are considering (there are 4096 possible 6mers).
5. Sort kmers by their corrected pvalues.
6. Write a file with the following columns for every kmer:
      kmer, frequency_in_enhanced, frequency_in_ctrl, enrichment (log2), uncorrected pvalue, corrected pvalue
7. Print the first 10 lines (the top 10 kmers sorted by pvalue) in your code block.      

Perhaps the best way to tackle this would be to write an independent function for each one of the numbered tasks above.  This keeps your code neat and readable.

In [ ]:
#PUT YOUR CODE FOR PROJECT 3 HERE

## Project 4

RNA molecules, being single stranded, have the ability to fold onto themselves into elaborate secondary structures by forming basepairs between nucleotides. These structures are often important for the biological function of the RNA.  Several modern algorithms exist for predicting RNA secondary structures from a given primary sequence.  One of these is called RNAfold.  When RNAfold is given the sequence of a noncoding RNA called 7SK (sequence below, make sure you scroll right),

>GGAUGUGAGGGCGAUCUGGCUGCGACAUCUGUCACCCCAUUGAUCGCCAGGGUUGAUUCGGCUGAUCUGGCUGGCUAGGCGGGUGUCCCCUUCCUCCCUCACCGCUCCAUGUGCGUCCCUCCCGAAGCUGCGCGCUCGGUCGAAGAGGACGACCAUCCCCGAUAGAGGAGGACCGGUCUUCGGUCAAGGGUAUACGAGUAGCUGCGCUCCCCUGCUAGAACCUCCAAACAAGCUCUCAAGGUCCAUUUGUAGGAGAACGUAGGGUAGUCAAGCUUCCAAGACUCCAGACACAUCCAAAUGAGGCGCUGCAUGUGGCAGUCUGCCUUUCUUUU

it predicts the following structure to be its most thermodynamically favored:

![7skstructure.png](7skstructure.png)

However, this is just one of many possible structures for this sequence. We might want to more accurately represent the whole ensemble of possible structures.  That is what we will do here.  During its computation, one of the things that RNAfold computes is the probability with each base is paired with every other base.  This is output as part of a text file.  That file for 7SK is [here](https://github.com/srinivasramachandran/Anschutz_python/blob/master/7skbpprob.txt).

Download this file. Parse it to get the basepairing probabilities for every possible nucleotide-nucleotide pair.  In this file, the base pair probability data begins with the line "%start of base pair probability data".  Every line after that contains four fields and is of the following form:

-------------------
position1 | position2 | **sqrt**(probability that position1 and position2 pair) | 'ubox'

-------------------

Probabilities of course range between 0 and 1. You will notice that not all possible pairs are listed in the file.  This could be for one of two reasons:

1. The probability that baseX and baseY interact is 0.
2. The probability that baseX and baseY interact has already been listed.  For example, position 1 and position 106 are listed as having an interaction probability of 0.003714731<sup>2</sup>.  The interaction between position 106 and position 1 will not be listed because it is the same as this and therefore redundant.

Construct a table that efficiently stores all of the basepair interaction data.  One possible structure is to have both column names (i) and row names (indicies) (j) be positions, and the value at i,j be their interaction probability.  

Perhaps the most efficient way to do this is to end up with a dictionary of the form {position : [list of all probabilities to other bases, IN THE ORDER OF THEIR POSITION]}.  This dictionary could be easily imported into a pandas dataframe. As a sanity check, in this dataframe, you can make sure that the probability at i,j is the same as the probability at j,i.

Using this table, make 2 plots.


### Plot 1

Plot the basepair probabilties as a heatmap where the rows and columns are positions, in order, and the color of the tile is the probability that the two positions are paired.  The [documentation](https://seaborn.pydata.org/generated/seaborn.heatmap.html) for seaborn heatmaps may be useful.


### Plot 2

For each position, plot the sum of all of the probabilities across all other positions.  This will be the probability that this position is paired at all.  Perhaps a [seaborn lineplot](https://seaborn.pydata.org/generated/seaborn.lineplot.html) would fit this data well. 



In [ ]:
# PUT YOUR CODE FOR PROJECT 4 HERE